In [1]:
# import os
# import time
#
# import matplotlib.pyplot as plt
import numpy as np
import numpy
from scipy.io import savemat, loadmat
# import torch
# from torch import nn
import torch
from torch.utils.data import DataLoader, Subset
# from torch.utils.tensorboard import SummaryWriter
import torchvision as tv
from torchvision import transforms
# import torchvision.utils as tvutils
from torchsummary import summary

# import cv2
# import PIL
# import matplotlib.pyplot as plt

from my_utils.vis import *
from my_utils import models
# from my_utils.utils import check_accuracy
from my_utils.data.dataset import raw_png_processor, raw_video_processor, MyDataset
from my_utils.data.loader import load_frames
from my_utils.data.preprocess import sub_mean, reduce

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)  # set default size of plots

# life save magic code
%load_ext autoreload
%autoreload 2

In [2]:
print(f"torch version: {torch.__version__}")
use_cuda = torch.cuda.is_available()
if use_cuda:
    GPU_nums = torch.cuda.device_count()
    GPU = torch.cuda.get_device_properties(0)
    print(f"There are {GPU_nums} GPUs in total.\nThe first GPU is: {GPU}")
    if '3060' in GPU.name:
        print(f"CUDA version: {torch.cuda_version}")
    else:
        print(f"CUDA version: {torch.version.cuda}")
device = torch.device(f"cuda:0" if use_cuda else "cpu")
print(f"Using {device} now!")

torch version: 1.12.0+cu113
There are 1 GPUs in total.
The first GPU is: _CudaDeviceProperties(name='NVIDIA A100-SXM4-80GB', major=8, minor=0, total_memory=81251MB, multi_processor_count=108)
CUDA version: 11.3
Using cuda:0 now!


In [11]:
tv_r21d_model = tv.models.video.r2plus1d_18()

In [16]:
my_r21d_model = models.NLOS_r21d(depths=(2,1), dims=(16, 32))

In [3]:
my_conv_model = models.NLOS_Conv(groups=2)

In [2]:
png_abs_dir = '/mnt/lustre/wangyihao/nlos_raw_pngs/train/Being hit_p0_r0_grey_tiling_07-16-15-49'
frames = load_frames(png_abs_dir, frame_range=(0, 64))
print(frames.shape)

torch.Size([64, 256, 256, 3])


In [3]:
frames_sub_mean = sub_mean(frames).numpy()

In [ ]:
frames_sub_mean

In [24]:
input_frames = torch.from_numpy(mat).permute(3, 0, 1, 2).unsqueeze(0)
input_frames.shape

torch.Size([1, 3, 64, 128, 128])

In [35]:
my_r21d_model((input_frames, torch.Tensor([0,], dtype=torch.float)))

TypeError: new() received an invalid combination of arguments - got (list, dtype=torch.dtype), but expected one of:
 * (*, torch.device device)
      didn't match because some of the keywords were incorrect: dtype
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, *, torch.device device)
 * (object data, *, torch.device device)


In [4]:
summary(my_conv_model.to(device), input_size=(6, 128, 128), batch_size=-1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 32, 64, 128]           4,704
       BatchNorm2d-2          [-1, 32, 64, 128]              64
         LeakyReLU-3          [-1, 32, 64, 128]               0
            Conv2d-4          [-1, 32, 64, 128]          25,088
       BatchNorm2d-5          [-1, 32, 64, 128]              64
         LeakyReLU-6          [-1, 32, 64, 128]               0
            Conv2d-7          [-1, 32, 64, 128]          25,088
       BatchNorm2d-8          [-1, 32, 64, 128]              64
         LeakyReLU-9          [-1, 32, 64, 128]               0
           Conv2d-10          [-1, 32, 64, 128]          25,088
      BatchNorm2d-11          [-1, 32, 64, 128]              64
        LeakyReLU-12          [-1, 32, 64, 128]               0
           Conv2d-13          [-1, 64, 64, 128]          50,176
      BatchNorm2d-14          [-1, 64, 

In [5]:
mat_file = loadmat(file_name='/mnt/lustre/wangyihao/nlos_raw_pngs/train/Lying down_p2_r0_white_tiling_07-15-03-16/video_128_N0.mat')
mat = mat_file['video']

In [22]:
mat[0].shape

(128, 128, 3)

In [21]:
torch.from_numpy(mat).permute(3, 0, 1, 2).shape

torch.Size([3, 64, 128, 128])

In [3]:
H = np.random.rand(128, 64, 3)
W = np.random.rand(128, 64, 3)
W.shape

(128, 64, 3)

In [8]:
np.concatenate(seq=(H, W), axis=2).shape

TypeError: concatenate() missing required argument 'seq' (pos 1)

In [11]:
np.dstack((H, W)).shape

(128, 64, 6)